In [1]:
import pandas as pd
import numpy as np
import re
import os

pd.set_option("display.max_columns", None)
pd.set_option('display.max_colwidth', None)

In [ ]:
pdfs = pd.read_csv('web-pdf.csv')
pdfs.head()

In [3]:
pdfs = pdfs.rename(columns={'contacto': 'nombre', 
                            'Equipment': 'amueblado', 
                            'precio_alquiler': 'precio',
                            'teléfono': 'telefono',
                            'baños': 'banos',
                            })
pdfs['tipo_transaccion'] = 'Alquiler'
pdfs['perfil'] = 'Particular'
pdfs['fecha'] = '2018-01-01'
pdfs['precio'] = pdfs['precio'].fillna(0).astype(int)
pdfs['superficie'] = pdfs['superficie'].fillna(0).astype(int)
pdfs['banos'] = pdfs['banos'].fillna(0).astype(int)
# Cambiar datos concretos de una columna, por ejemplo, cambiar el valor de 'precio' donde 'ciudad' es 'Murcia'
# Reemplazar los valores de 'No especificado' en la columna 'habitaciones' por 0
pdfs.loc[pdfs['habitaciones'] == 'No especificado', 'habitaciones'] = 0
# Cambiar el tipo de la columna 'habitaciones' a entero
pdfs['habitaciones'] = pdfs['habitaciones'].astype(int)



# Definir la función para extraer los datos
def extraer_datos(row):
    # Usar una expresión regular para capturar "tipo de vivienda", "street" y "place"
    match = re.search(r'Alquiler de (.+?) en (.*?, \d+), (.*?)\.pdf', row)
    if match:
        tipo = match.group(1)  # Captura el tipo de vivienda (piso, casa, etc.)
        calle = match.group(2)         # Captura la calle y número
        barrio = match.group(3)          # Captura el barrio y la ciudad
        return pd.Series([tipo, calle, barrio])
    else:
        return pd.Series([None, None, None])  # Si no coincide, devuelve None

# Aplicar la función al DataFrame y crear las nuevas columnas
pdfs[['tipo', 'calle', 'barrio']] = pdfs['pdf'].apply(extraer_datos)

In [ ]:
euspay = pd.read_csv('euspay-murcia.csv')
euspay.drop("Unnamed: 0", axis=1, inplace=True)
euspay.info()

In [6]:
euspay = euspay.rename(columns={'id_source': 'id_inmueble',
                                'name_contact': 'nombre',
                                'telephone': 'telefono',
                                'type_operation': 'tipo_transaccion',
                                'property': 'tipo',
                                'profile': 'perfil',
                                'street': 'calle',
                                'place': 'barrio',
                                'meters': 'superficie',
                                'rooms': 'habitaciones',
                                'orientation': 'orientacion',
                                'price': 'precio',
                                'date_scrape': 'fecha',
                                })

euspay = euspay.drop(columns={'id', 'ask_advertiser', 'date', 'source'})

In [ ]:
euspay.info()

In [ ]:
bd_pdf = pdfs[['nombre', 'perfil', 'telefono', 'tipo_transaccion', 'tipo', 'calle', 'barrio', 'superficie', 'habitaciones', 'banos',  ]]


In [ ]:
pdfs.info()

In [ ]:
df_merged = pd.merge(pdfs, euspay, on='id_inmueble')

In [2]:
particulares = pd.read_csv('../../soporte/particulares/particulares_murcia.csv')
particulares= particulares.drop(['Unnamed: 0'], axis=1)
particulares = particulares.rename(columns={'date_scrape':'fecha'})
# Separar la información de la columna 'barrio' en dos columnas: 'barrio' y 'ciudad'
# Utiliza 'expand=True' y luego 'n=1' para asegurarse de que la división tenga dos partes (rellenará con NaN si no hay coma)
particulares[['barrio', 'ciudad']] = particulares['barrio'].str.split(',', n=1, expand=True)

# Limpiar espacios en ambas columnas
particulares['barrio'] = particulares['barrio'].str.strip()
particulares['ciudad'] = particulares['ciudad'].str.strip().fillna('Murcia')

# Unir las columnas 'calle', 'barrio' y 'ciudad' para crear la nueva columna 'localizacion'
particulares['localizacion'] = particulares['calle'] + ', ' + particulares['barrio'] + ', ' + particulares['ciudad']

# Eliminar filas donde la columna 'telefono' esté vacía
particulares = particulares.dropna(subset=['telefono'])

particulares.head(5) 

,nombre,telefono,tipo_transaccion,tipo,tipo_contacto,calle,barrio,superficie,habitaciones,planta,caracteristicas,precio,id_inmueble,fecha,banos,estado,armarios,orientacion,amueblado,exterior_interior,ascensor,ciudad,localizacion
0,Antonio,633 860 715,Alquiler,Piso,Particular,"calle Valle de Leiva, 5",Puente Tocinos,110,3,1,"110 m² construidos, 90 m² útiles 3 habitaciones 2 baños Segunda mano/buen estado Armarios empotrados Orientación sur Amueblado y cocina equipada Certificación energética: no indicado",495,26861942,2019-11-29,2,Segunda mano/buen estado,Sí,sur,Sí,Exterior,Sí,Murcia,"calle Valle de Leiva, 5, Puente Tocinos, Murcia"
1,pascual,968 264 704,Alquiler,Piso,Particular,"calle Escuelas, 41",Alquerías,116,3,1,116 m² construidos 3 habitaciones 2 baños Balcón Plaza de garaje incluida en el precio Segunda mano/buen estado Armarios empotrados Cocina equipada y casa sin amueblar Calefacción individual Certificación energética: (IPE no indicado) Vivienda apta para su uso por personas con movilidad reducida,490,83207117,2019-11-29,2,Segunda mano/buen estado,Sí,No indicado,No,Exterior,Sí,Murcia,"calle Escuelas, 41, Alquerías, Murcia"
2,Ender Fuenmayor,627 226 825,Alquiler,Piso,Particular,"calle Escritor Francisco Aleman Sainz, 2",Espinardo,95,3,5,"95 m² construidos, 73 m² útiles 3 habitaciones 2 baños Plaza de garaje incluida en el precio Segunda mano/buen estado Armarios empotrados Cocina equipada y casa sin amueblar Calefacción central Certificación energética: no indicado Acceso adaptado a personas con movilidad reducida Vivienda apta para su uso por personas con movilidad reducida",500,83714401,2022-06-14,2,Segunda mano/buen estado,Sí,No indicado,No,Exterior,Sí,Murcia,"calle Escritor Francisco Aleman Sainz, 2, Espinardo, Murcia"
3,Alberto,659 772 242,Alquiler,Piso,Particular,"carril Cuevas, 14",Cabezo de Torres,100,3,1,100 m² construidos 3 habitaciones 2 baños Plaza de garaje incluida en el precio Segunda mano/buen estado Trastero Orientación este Cocina equipada y casa sin amueblar Calefacción individual: Bomba de frío/calor Certificación energética: no indicado,500,83714094,2019-11-29,2,Segunda mano/buen estado,No,este,No,Exterior,Sí,Murcia,"carril Cuevas, 14, Cabezo de Torres, Murcia"
4,Timoteo Gimenez,606 069 412,Alquiler,Ático,Particular,"calle Jardines, 1",Alhama de Murcia,90,2,0,"90 m² construidos, 80 m² útiles 2 habitaciones 1 baño Terraza Plaza de garaje incluida en el precio Segunda mano/buen estado Armarios empotrados Orientación sur Amueblado y cocina equipada Certificación energética: no indicado",350,1675052,2019-11-29,1,Segunda mano/buen estado,Sí,sur,Sí,Exterior,No,Murcia,"calle Jardines, 1, Alhama de Murcia, Murcia"


In [3]:
# Leer archivo CSV
web = pd.read_csv("../../web_2/data/inmuebles.csv")

# Eliminar columnas innecesarias
web = web.drop(['url', 'superficie_util', 'referencia', 'planta', 'zona', 'titulo',
                'calefaccion', 'construccion'], axis=1)

# Renombrar columnas
web = web.rename(columns={'anunciante': 'tipo_contacto', 'tlf': 'telefono', 'tipo': 'tipo_transaccion', 'id': 'id_inmueble'})

# Limpiar y convertir la columna 'precio'
web['precio'] = web['precio'].str.replace('.', '').str.replace('€', '').str.strip()
web['precio'] = pd.to_numeric(web['precio'], errors='coerce').fillna(0.0)

# Limpiar la columna 'precio_metro'
web['precio_metro'] = web['precio_metro'].str.replace('.', ',').str.strip()

# Convertir las columnas 'habitaciones', 'baños', y 'construccion' a numéricas
web['habitaciones'] = pd.to_numeric(web['habitaciones'], errors='coerce').fillna(0).astype(int)
web["baños"] = pd.to_numeric(web['baños'], errors='coerce').fillna(0).astype(int)
# web["construccion"] = pd.to_numeric(web['construccion'], errors='coerce').fillna(0).astype(int)

# Extraer información de la columna 'caracteristicas'

# Extraer número de baños (int)
web['banos'] = web['caracteristicas'].str.extract(r'(\d+) baños?', expand=False).astype('float').fillna(0).astype('int')

# Extraer estado del inmueble
web['estado'] = web['caracteristicas'].str.extract(r'(Segunda mano\/buen estado|Nueva construcción|A reformar)', expand=False).fillna('No indicado')

# Extraer si tiene armarios empotrados (Sí o No)
web['armarios'] = web['caracteristicas'].str.contains('Armarios empotrados', case=False, na=False).map({True: 'Sí', False: 'No'})

# Extraer orientación (norte, sur, este, oeste)
web['orientacion'] = web['caracteristicas'].str.extract(r'Orientación (\w+)', expand=False).fillna('No indicado')

# Extraer si está amueblado (Sí o No)
web['amueblado'] = web['caracteristicas'].str.contains('Amueblado', case=False, na=False).map({True: 'Sí', False: 'No'})

# Extraer si es exterior o interior
web['exterior_interior'] = web['caracteristicas'].str.extract(r'(exterior|interior)', expand=False).fillna('No indicado').str.capitalize()

# Extraer si tiene ascensor (Sí o No)
web['ascensor'] = web['caracteristicas'].str.contains('con ascensor', case=False, na=False).map({True: 'Sí', False: 'No'})

# Quitar el prefijo '34' si el número empieza con este prefijo
# web['telefono'] = web['telefono'].str.replace(r'^(\+34|0034|34)\s*', '', regex=True)

# Eliminar filas donde la columna 'telefono' esté vacía después del proceso
web = web.dropna(subset=['telefono'])

# Extraer solo la parte de la fecha (sin la hora)
web['fecha'] = web['fecha'].str.split(' ', n=1, expand=True)[0]

# Mostrar las primeras 5 filas del DataFrame actualizado
web.head(5)


,id_inmueble,tipo_transaccion,calle,barrio,ciudad,localizacion,precio,precio_metro,caracteristicas,habitaciones,baños,tipo_contacto,nombre,telefono,fecha,superficie,terraza,garaje,estado,armarios,trastero,orientacion,ascensor,banos,amueblado,exterior_interior
0,106607970,Ventas,Calle Zamora,El Ranero-San Basilio,Murcia,"Calle Zamora, Barrio El Ranero-San Basilio, Distrito Norte, Murcia, Área de Murcia, Murcia",97900,"2,967","['33 m² construidos, 30 m² útiles', 'Sin habitación', '1 baño', 'Segunda mano/buen estado', 'Construido en 2025', 'No dispone de calefacción', 'Bajo exterior', 'Sin ascensor']",0,1,Profesional,Kakebo Inmobiliaria,34868350168,2024-11-20,33,NaN,NaN,Segunda mano/buen estado,No,NaN,No indicado,No,1,No,Exterior
1,106607801,Ventas,"Calle Almohajar, 8",El Carmen,Murcia,"Calle Almohajar, 8, Barrio El Carmen, Distrito Sur, Murcia, Área de Murcia, Murcia",425000,"3,400","['125 m² construidos, 120 m² útiles', '4 habitaciones', '2 baños', 'Terraza', 'Plaza de garaje incluida en el precio', 'Segunda mano/buen estado', 'Armarios empotrados', 'Trastero', 'Orientación norte, sur', 'Construido en 1996', 'Calefacción individual: Gas natural', 'Solo acceso exterior adaptado para personas con movilidad reducida', 'Planta 4ª exterior', 'Con ascensor']",4,2,Particular,Antonio,34607500990,2024-11-20,125,Terraza,Plaza de garaje,Segunda mano/buen estado,Sí,Trastero,norte,Sí,2,No,Exterior
2,106607969,Ventas,"Avenida Juan Carlos I, 23",Santa María de Gracia,Murcia,"Avenida Juan Carlos I, 23, Barrio Santa María de Gracia, Distrito Norte, Murcia, Área de Murcia, Murcia",140000,"2,333","['Alquilada, con inquilinos', '60 m² construidos, 45 m² útiles', '1 habitación', '1 baño', 'Balcón', 'Segunda mano/buen estado', 'Armarios empotrados', 'Orientación oeste', 'Construido en 1992', 'Solo acceso exterior adaptado para personas con movilidad reducida', 'Planta 2ª exterior', 'Con ascensor']",0,1,Profesional,LIDERCASA INTERNATIONAL,34868355695,2024-11-20,60,Balcón,NaN,Segunda mano/buen estado,Sí,NaN,oeste,Sí,1,No,Exterior
3,106603650,Ventas,Calle Gibraltar,Algezares,Murcia,"Calle Gibraltar, Barrio Algezares, Distrito Pedanías Este, Murcia, Área de Murcia, Murcia",151000,926,"['163 m² construidos, 115 m² útiles', '3 habitaciones', '2 baños', 'Segunda mano/buen estado', 'Construido en 2007', 'No dispone de calefacción', 'Planta 3ª exterior', 'Con ascensor']",3,2,Profesional,Soluciones Inmobiliaria Trial3 S.l.,34854780070,2024-11-20,163,NaN,NaN,Segunda mano/buen estado,No,NaN,No indicado,Sí,2,No,Exterior
4,106607158,Alquiler,NaN,Javalí Nuevo,Murcia,"Barrio Javalí Nuevo, Distrito Pedanías Oeste, Murcia, Área de Murcia, Murcia",800,"9,88","['81 m² construidos', '3 habitaciones', '1 baño', 'Segunda mano/buen estado', 'Armarios empotrados', 'Construido en 1974', 'Amueblado y cocina equipada', 'Bajo']",3,1,Profesional,GRUPO INMOBILIARIO MARAL carratalá garcía,34965025065,2024-11-20,81,NaN,NaN,Segunda mano/buen estado,Sí,NaN,No indicado,No,1,Sí,No indicado


In [7]:
web['tipo_transaccion'].value_counts()

tipo_transaccion
Ventas      2763
Alquiler    1232
Name: count, dtype: int64

In [9]:
web['tipo_contacto'].value_counts()

tipo_contacto
Profesional    3026
Particular      668
Promotora        80
Agente Pro        9
Name: count, dtype: int64

In [8]:
particulares['tipo_transaccion'].value_counts()

tipo_transaccion
Alquiler    1318
Name: count, dtype: int64

In [4]:
# Unir los DataFrames 'web' y 'particulares' utilizando 'id_inmueble' como clave
total_particulares = pd.concat([particulares, web], ignore_index=True)
total_particulares

,nombre,telefono,tipo_transaccion,tipo,tipo_contacto,calle,barrio,superficie,habitaciones,planta,caracteristicas,precio,id_inmueble,fecha,banos,estado,armarios,orientacion,amueblado,exterior_interior,ascensor,ciudad,localizacion,precio_metro,baños,terraza,garaje,trastero
0,Antonio,633 860 715,Alquiler,Piso,Particular,"calle Valle de Leiva, 5",Puente Tocinos,110,3,1.0,"110 m² construidos, 90 m² útiles 3 habitaciones 2 baños Segunda mano/buen estado Armarios empotrados Orientación sur Amueblado y cocina equipada Certificación energética: no indicado",495,26861942,2019-11-29,2,Segunda mano/buen estado,Sí,sur,Sí,Exterior,Sí,Murcia,"calle Valle de Leiva, 5, Puente Tocinos, Murcia",NaN,NaN,NaN,NaN,NaN
1,pascual,968 264 704,Alquiler,Piso,Particular,"calle Escuelas, 41",Alquerías,116,3,1.0,116 m² construidos 3 habitaciones 2 baños Balcón Plaza de garaje incluida en el precio Segunda mano/buen estado Armarios empotrados Cocina equipada y casa sin amueblar Calefacción individual Certificación energética: (IPE no indicado) Vivienda apta para su uso por personas con movilidad reducida,490,83207117,2019-11-29,2,Segunda mano/buen estado,Sí,No indicado,No,Exterior,Sí,Murcia,"calle Escuelas, 41, Alquerías, Murcia",NaN,NaN,NaN,NaN,NaN
2,Ender Fuenmayor,627 226 825,Alquiler,Piso,Particular,"calle Escritor Francisco Aleman Sainz, 2",Espinardo,95,3,5.0,"95 m² construidos, 73 m² útiles 3 habitaciones 2 baños Plaza de garaje incluida en el precio Segunda mano/buen estado Armarios empotrados Cocina equipada y casa sin amueblar Calefacción central Certificación energética: no indicado Acceso adaptado a personas con movilidad reducida Vivienda apta para su uso por personas con movilidad reducida",500,83714401,2022-06-14,2,Segunda mano/buen estado,Sí,No indicado,No,Exterior,Sí,Murcia,"calle Escritor Francisco Aleman Sainz, 2, Espinardo, Murcia",NaN,NaN,NaN,NaN,NaN
3,Alberto,659 772 242,Alquiler,Piso,Particular,"carril Cuevas, 14",Cabezo de Torres,100,3,1.0,100 m² construidos 3 habitaciones 2 baños Plaza de garaje incluida en el precio Segunda mano/buen estado Trastero Orientación este Cocina equipada y casa sin amueblar Calefacción individual: Bomba de frío/calor Certificación energética: no indicado,500,83714094,2019-11-29,2,Segunda mano/buen estado,No,este,No,Exterior,Sí,Murcia,"carril Cuevas, 14, Cabezo de Torres, Murcia",NaN,NaN,NaN,NaN,NaN
4,Timoteo Gimenez,606 069 412,Alquiler,Ático,Particular,"calle Jardines, 1",Alhama de Murcia,90,2,0.0,"90 m² construidos, 80 m² útiles 2 habitaciones 1 baño Terraza Plaza de garaje incluida en el precio Segunda mano/buen estado Armarios empotrados Orientación sur Amueblado y cocina equipada Certificación energética: no indicado",350,1675052,2019-11-29,1,Segunda mano/buen estado,Sí,sur,Sí,Exterior,No,Murcia,"calle Jardines, 1, Alhama de Murcia, Murcia",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5308,The Simple Rent,34868350665,Alquiler,NaN,Profesional,Avenida Don Juan de Borbón,Vista Alegre,70,2,NaN,"['70 m² construidos, 66 m² útiles', '2 habitaciones', '1 baño', 'Plaza de garaje incluida en el precio', 'Segunda mano/buen estado', 'Armarios empotrados', 'Trastero', 'Amueblado y cocina equipada', 'Planta 3ª exterior', 'Con ascensor']",770,1473835,2024-10-18,1,Segunda mano/buen estado,Sí,No indicado,Sí,Exterior,Sí,Murcia,"Avenida Don Juan de Borbón, Barrio Vista Alegre, Distrito Norte, Murcia, Área de Murcia, Murcia","11,00",1.0,NaN,Plaza de garaje,Trastero
5309,Mari Carmen,34629278554,Alquiler,NaN,Particular,"Calle tornel, 1",Los Dolores,100,3,NaN,"['100 m² construidos, 85 m² útiles', '3 habitaciones', '2 baños', 'Plaza de garaje incluida en el precio', 'Segunda mano/buen estado', 'Armarios empotrados', 'Trastero', 'Orientación sur, oeste', 'Amueblado y cocina equipada', 'Planta 2ª exterior', 'Con ascensor']",700,34614749,2024-10-18,2,Segunda mano/buen estado,Sí,sur,Sí,Exterior,Sí,Murcia,"Calle tornel, 1, Barrio Los Dolore

In [5]:
total_particulares = total_particulares[total_particulares['tipo_contacto'] == 'Particular']
total_particulares


,nombre,telefono,tipo_transaccion,tipo,tipo_contacto,calle,barrio,superficie,habitaciones,planta,caracteristicas,precio,id_inmueble,fecha,banos,estado,armarios,orientacion,amueblado,exterior_interior,ascensor,ciudad,localizacion,precio_metro,baños,terraza,garaje,trastero
0,Antonio,633 860 715,Alquiler,Piso,Particular,"calle Valle de Leiva, 5",Puente Tocinos,110,3,1.0,"110 m² construidos, 90 m² útiles 3 habitaciones 2 baños Segunda mano/buen estado Armarios empotrados Orientación sur Amueblado y cocina equipada Certificación energética: no indicado",495,26861942,2019-11-29,2,Segunda mano/buen estado,Sí,sur,Sí,Exterior,Sí,Murcia,"calle Valle de Leiva, 5, Puente Tocinos, Murcia",NaN,NaN,NaN,NaN,NaN
1,pascual,968 264 704,Alquiler,Piso,Particular,"calle Escuelas, 41",Alquerías,116,3,1.0,116 m² construidos 3 habitaciones 2 baños Balcón Plaza de garaje incluida en el precio Segunda mano/buen estado Armarios empotrados Cocina equipada y casa sin amueblar Calefacción individual Certificación energética: (IPE no indicado) Vivienda apta para su uso por personas con movilidad reducida,490,83207117,2019-11-29,2,Segunda mano/buen estado,Sí,No indicado,No,Exterior,Sí,Murcia,"calle Escuelas, 41, Alquerías, Murcia",NaN,NaN,NaN,NaN,NaN
2,Ender Fuenmayor,627 226 825,Alquiler,Piso,Particular,"calle Escritor Francisco Aleman Sainz, 2",Espinardo,95,3,5.0,"95 m² construidos, 73 m² útiles 3 habitaciones 2 baños Plaza de garaje incluida en el precio Segunda mano/buen estado Armarios empotrados Cocina equipada y casa sin amueblar Calefacción central Certificación energética: no indicado Acceso adaptado a personas con movilidad reducida Vivienda apta para su uso por personas con movilidad reducida",500,83714401,2022-06-14,2,Segunda mano/buen estado,Sí,No indicado,No,Exterior,Sí,Murcia,"calle Escritor Francisco Aleman Sainz, 2, Espinardo, Murcia",NaN,NaN,NaN,NaN,NaN
3,Alberto,659 772 242,Alquiler,Piso,Particular,"carril Cuevas, 14",Cabezo de Torres,100,3,1.0,100 m² construidos 3 habitaciones 2 baños Plaza de garaje incluida en el precio Segunda mano/buen estado Trastero Orientación este Cocina equipada y casa sin amueblar Calefacción individual: Bomba de frío/calor Certificación energética: no indicado,500,83714094,2019-11-29,2,Segunda mano/buen estado,No,este,No,Exterior,Sí,Murcia,"carril Cuevas, 14, Cabezo de Torres, Murcia",NaN,NaN,NaN,NaN,NaN
4,Timoteo Gimenez,606 069 412,Alquiler,Ático,Particular,"calle Jardines, 1",Alhama de Murcia,90,2,0.0,"90 m² construidos, 80 m² útiles 2 habitaciones 1 baño Terraza Plaza de garaje incluida en el precio Segunda mano/buen estado Armarios empotrados Orientación sur Amueblado y cocina equipada Certificación energética: no indicado",350,1675052,2019-11-29,1,Segunda mano/buen estado,Sí,sur,Sí,Exterior,No,Murcia,"calle Jardines, 1, Alhama de Murcia, Murcia",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5306,Ramón,34607583501,Alquiler,NaN,Particular,"Avenida Floridablanca, 62",El Carmen,118,0,NaN,"['118 m² construidos, 105 m² útiles', '1 habitación', '1 baño', 'Balcón', 'Segunda mano/buen estado', 'Armarios empotrados', 'Cocina equipada y casa sin amueblar', 'Calefacción individual: Eléctrica', 'Planta 4ª exterior', 'Con ascensor']",800,106308045,2024-10-18,1,Segunda mano/buen estado,Sí,No indicado,No,Exterior,Sí,Murcia,"Avenida Floridablanca, 62, Barrio El Carmen, Distrito Sur, Murcia, Área de Murcia, Murcia","6,78",1.0,Balcón,NaN,NaN
5309,Mari Carmen,34629278554,Alquiler,NaN,Particular,"Calle tornel, 1",Los Dolores,100,3,NaN,"['100 m² construidos, 85 m² útiles', '3 habitaciones', '2 baños', 'Plaza de garaje incluida en el precio', 'Segunda mano/buen estado', 'Armarios empotrados', 'Trastero', 'Orientación sur, oeste', 'Amueblado y cocina equipada', 'Planta 2ª exterior', 'Con ascensor']",700,34614749,2024-10-18,2,Segunda mano/buen estado,Sí,sur,Sí,Exterior,Sí,Murcia,"Calle tornel, 1, Barrio Los Dolores, Distrito Pedanías Este, Mu

In [6]:
total_particulares.to_excel('particulares_alquiler.xlsx')

In [2]:
web2 = pd.read_csv("../../web_2/data/inmuebles.csv")
web2.head(1)

,id,tipo,titulo,calle,barrio,zona,ciudad,localizacion,precio,precio_metro,caracteristicas,habitaciones,baños,referencia,anunciante,nombre,tlf,url,fecha,superficie,superficie_util,terraza,garaje,estado,armarios,trastero,orientacion,calefaccion,planta,ascensor,construccion
0,106607970,Ventas,Estudio en venta en calle Zamora,Calle Zamora,El Ranero-San Basilio,Norte,Murcia,"Calle Zamora, Barrio El Ranero-San Basilio, Distrito Norte, Murcia, Área de Murcia, Murcia",97.900€,2.967,"['33 m² construidos, 30 m² útiles', 'Sin habitación', '1 baño', 'Segunda mano/buen estado', 'Construido en 2025', 'No dispone de calefacción', 'Bajo exterior', 'Sin ascensor']",NaN,1.0,ESTUDIOSELRANERO,Profesional,Kakebo Inmobiliaria,34868350168,https://www.idealista.com/inmueble/106607970/,2024-11-20 09:42:02,33,NaN,NaN,NaN,Segunda mano/buen estado,NaN,NaN,NaN,NaN,NaN,No,2025.0


In [5]:
comunes= web2[(web2['tipo'] == 'Alquiler') & (web2['anunciante'] == 'Particular')]
comunes

,id,tipo,titulo,calle,barrio,zona,ciudad,localizacion,precio,precio_metro,caracteristicas,habitaciones,baños,referencia,anunciante,nombre,tlf,url,fecha,superficie,superficie_util,terraza,garaje,estado,armarios,trastero,orientacion,calefaccion,planta,ascensor,construccion
30,105745528,Alquiler,"Estudio en calle Salitre, 35","Calle Salitre, 35",La Alberca,Este,Murcia,"Calle Salitre, 35, Barrio La Alberca, Distrito Pedanías Este, Murcia, Área de Murcia, Murcia",550,"11,96","['46 m² construidos', 'Sin habitación', '1 baño', 'Balcón', 'Plaza de garaje incluida en el precio', 'Segunda mano/buen estado', 'Armarios empotrados', 'Trastero', 'Orientación norte', 'Amueblado y cocina equipada', 'Planta 1ª exterior', 'Con ascensor']",NaN,1.0,105745528,Particular,Raquel,-1,https://www.idealista.com/inmueble/105745528/,2024-11-19 20:00:04,46,NaN,Balcón,Plaza de garaje,Segunda mano/buen estado,Armarios empotrados,Trastero,norte,NaN,NaN,Sí,NaN
47,105738067,Alquiler,"Piso en calle Magallanes, 4","Calle Magallanes, 4",La Flota,Norte,Murcia,"Calle Magallanes, 4, Barrio La Flota, Distrito Norte, Murcia, Área de Murcia, Murcia",750,"13,64","['55 m² construidos', '1 habitación', '2 baños', 'Segunda mano/buen estado', 'Orientación sur', 'Amueblado y cocina equipada', 'Bajo exterior', 'Sin ascensor']",NaN,2.0,105738067,Particular,Miguel Giménez,34601294735,https://www.idealista.com/inmueble/105738067/,2024-11-19 16:00:04,55,NaN,NaN,NaN,Segunda mano/buen estado,NaN,NaN,sur,NaN,NaN,No,NaN
73,106589596,Alquiler,"Piso en calle Mayor, 54","Calle Mayor, 54",La Ñora,Este,Murcia,"Calle Mayor, 54, Barrio La Ñora, Distrito Pedanías Oeste, Murcia, Área de Murcia, Murcia",600,"30,00","['20 m² construidos, 15 m² útiles', '30 habitaciones', '30 baños', 'Terraza y balcón', 'Segunda mano/buen estado', 'Armarios empotrados', 'Orientación este', 'Amueblado y cocina equipada', 'Planta 2ª exterior', 'Con ascensor']",30.0,30.0,106589596,Particular,Teresa,34679403646,https://www.idealista.com/inmueble/106589596/,2024-11-19 12:00:04,20,NaN,Terraza,NaN,Segunda mano/buen estado,Armarios empotrados,NaN,este,NaN,NaN,Sí,NaN
82,106593109,Alquiler,"Dúplex en calle Santiago, 5","Calle Santiago, 5",La Alberca,Este,Murcia,"Calle Santiago, 5, Barrio La Alberca, Distrito Pedanías Este, Murcia, Área de Murcia, Murcia",1.000,"6,54","['153 m² construidos', '2 habitaciones', '3 baños', 'Terraza y balcón', 'Plaza de garaje por 100 €/mes adicionales', 'Segunda mano/buen estado', 'Orientación sur', 'Cocina equipada y casa sin amueblar', 'Planta 3ª exterior', 'Con ascensor']",2.0,3.0,106593109,Particular,Pepe,34696061081,https://www.idealista.com/inmueble/106593109/,2024-11-19 00:00:04,153,NaN,Terraza,Plaza de garaje,Segunda mano/buen estado,NaN,NaN,sur,NaN,NaN,Sí,NaN
83,106592763,Alquiler,"Estudio en carretera de la Fuensanta, 83","Carretera de la Fuensanta, 83",San Benito-Patiño,Este,Murcia,"Carretera de la Fuensanta, 83, Barrio San Benito-Patiño, Distrito Pedanías Este, Murcia, Área de Murcia, Murcia",540,"9,00","['60 m² construidos, 55 m² útiles', 'Sin habitación', '1 baño', 'Terraza', 'Segunda mano/buen estado', 'Armarios empotrados', 'Orientación este', 'Cocina equipada y casa sin amueblar', 'Planta 1ª exterior', 'Sin ascensor']",NaN,1.0,106592763,Particular,Pascual Atenza,34635910068,https://www.idealista.com/inmueble/106592763/,2024-11-19 00:00:04,60,NaN,Terraza,NaN,Segunda mano/buen estado,Armarios empotrados,NaN,este,NaN,NaN,No,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3988,106308045,Alquiler,"Piso en avenida Floridablanca, 62","Avenida Floridablanca, 62",El Carmen,Sur,Murcia,"Avenida Floridablanca, 62, Barrio El Carmen, Distrito Sur, Murcia, Área de Murcia, Murcia",800,"6,78","['118 m² construidos, 105 m² útiles', '1 habitación', '1 baño', 'Balcón', 'Segunda mano/buen estado', 'Armarios empotrados', 'Cocina equipada y casa sin amueblar', 'Calefacción individual: Eléctrica', 'Planta 4ª exterior'

In [4]:
web2['tipo'].value_counts()

tipo
Ventas      2763
Alquiler    1232
Name: count, dtype: int64